In [1]:
import re
import numpy as np
import pandas as pd

import nltk
from nltk import *

In [2]:
#read out the CXR report dataset csv file

df_raw = pd.read_excel('data//530k DB.xlsx').sample(frac=1)


In [3]:
df_raw

,ACCNO,EXAMDATE,EXAMNAME,PATID,FILMAGE,CHSEX,ORDERFROM,ORDERDEPT,WARD,CONFIRMDR,CONFIRMDRID,BODYPART,MACHINEID,ITEMNAME,IMAGECOUNT,FINDINGS,IMPRESSIONS
59139,RA06A19809720004,2017/10/23 上午 08:30:59,Chest PA,R72581,26,0,H,高檢,NaN,廖揚振,88682.0,CHEST,DR1,1,1.0,Chest PA shows no definite lung lesion and no...,no definite abnormal findings.
241949,RA07507165340002,2018/5/7 上午 09:25:31,Chest PA,Y86140,43,0,H,健康管理中心,NaN,曾水權,2863.0,CHEST,DR4,1,1.0,Film of the regular chest:Routine.$$The heart ...,No significant findings of the chest but exagg...
381750,RA06430728640001,2017/4/30 下午 03:44:14,Chest AP,K55807,91,0,OPDE,一般內科,NaN,賴威任,94371.0,CHEST,DR7,1,1.0,Supine chest AP view was taken$Heart size eval...,gaseous distension of the bowel loops
77622,RA01210631660879,2012/2/10 上午 03:50:24,Chest,S27341,45,1,IPD,一般外科,05D151,陳慧燕,89527.0,CHEST,CR2,1,NaN,Left lung opacities. Elevation of right hemidi...,Left lung opacities. R/O left pleural effusion...
281346,RA02128119720011,2013/1/28 上午 05:54:12,Chest,T11952,71,0,OPDE,一般外科,NaN,曾旭明,89085.0,CHEST,DR5,1,NaN,CXR shows cardiomegaly and tortuous aorta with...,Cardiomegaly.$Atherosclerosis.
361420,RA05827982000010,2016/8/27 上午 08:41:50,Chest PA,A77482,62,1,OPDE,一般內科,NaN,賴彥君,86896.0,CHEST,DR7,1,1.0,Erect PA projection of the chest film is taken...,Peribronchial/interstitial infiltrates are not...
255573,RA07602125530002,2018/6/2 上午 10:50:45,Chest PA,769184,45,1,OPDR,心臟血管內科,NaN,劉武翹,94348.0,CHEST,DR8,1,1.0,Chest film shows mild left ventricular enlarge...,no active lung lesion.
419605,RA07605751860023,2018/6/5 下午 11:07:34,Chest AP (Portable),H67400,77,1,OPDE,一般內科,NaN,黃俊傑,89088.0,CHEST,CR2,1,1.0,Chest X-ray film shows large opacity at right ...,suspected pneumonia at right thorax.$retention...
306398,RA03A30795961820,2014/10/30 上午 01:23:19,Chest,101,87,1,IPD,胸腔內科,03D201,張永強,86196.0,CHEST,CR2,1,NaN,The supine AP chest radiograph shows: $* Persi...,1. Persistence of pneumonia at bilateral lower...
75494,RA04B14543520001,2015/11/14 下午 12:58:54,Chest PA,W01311,59,0,OPDR,一般外科,NaN,郭冠宏,88780.0,CHEST,DR3,1,1.0,CXR of PA view:$$> Normal heart size and tortu...,> Prominance of bilateral lung markings. $


In [4]:
def text_clean_up(text, remove_stopwords = False, stem_words = False, lemmatization= True):
    
    ## Convert words to lower case

    text = str(text).lower()
    
    ## Remove puncuation and replace common abbreviations
    text = re.sub('s/p','status post',text)
    text = re.sub('r/o','rule out',text)
    text = re.sub('r/i','rule in',text)
    text = re.sub('f/u','follow up',text)
    text = re.sub('-','',text)
    text = re.sub(r'[^a-zA-Z0-9_]',' ',text)
    text = text.split()
    
    # Optionally, remove stop words
    if remove_stopwords:
        
        stops = set(stopwords.words("english"))
        
        text = [w for w in text if not w in stops]
        
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        
        snowball = SnowballStemmer("english", ignore_stopwords=True)
        
        stemmed_words = [snowball.stem(word) for word in text]
        
        text = " ".join(stemmed_words)
        
    # WordNet lemmatizer only removes affixes if the resulting word is in its dictionary    
    if lemmatization:
        
        lemmatizer = nltk.WordNetLemmatizer()        
        
        lemmatized_words = [lemmatizer.lemmatize(word) for word in text]
        
        text = " ".join(lemmatized_words)

    return text

In [5]:
acc = df_raw['ACCNO'].values
findings = df_raw['FINDINGS'].map(lambda x: text_clean_up(x)).values
PE_labels = np.zeros(findings.shape[0],dtype = int)

df_report_only = pd.DataFrame({'ACCNO':acc,'Report': findings,'PE Label': PE_labels})

In [6]:
#Analyze the whole report text corpus (just exploratory not helpful in this task I believe)

report_corpus = nltk.Text(" ".join(findings).split())

total_vocab_count = len(set(report_corpus))

lexical_diversity = len(set(report_corpus)) / len(report_corpus)

freq_rank = 50

print("\n Word count of the complete report text corpus: {}"
      .format(len(report_corpus)))

print("\n Vocabulary count of the complete report text corpus: {}"
      .format(total_vocab_count))

print("\n Lexical diversity (Lexical richness of the text) : \n {}"
      .format(lexical_diversity))

print("\n {} most common vocabs: \n\n {} "
      .format(freq_rank, FreqDist(report_corpus).most_common(freq_rank)))


 Word count of the complete report text corpus: 13471049

 Vocabulary count of the complete report text corpus: 15925

 Lexical diversity (Lexical richness of the text) : 
 0.0011821648039436275

 50 most common vocabs: 

 [('of', 483438), ('the', 466103), ('lung', 455058), ('no', 403730), ('and', 383580), ('chest', 341281), ('is', 305884), ('show', 294332), ('in', 267722), ('normal', 265524), ('heart', 250178), ('size', 248655), ('noted', 206496), ('with', 198992), ('are', 198566), ('lesion', 197786), ('bilateral', 183872), ('at', 183018), ('pa', 178708), ('right', 159215), ('definite', 152393), ('left', 140501), ('status', 137331), ('post', 137259), ('view', 122951), ('change', 117739), ('both', 112482), ('field', 107914), ('thoracic', 106893), ('mild', 104609), ('aorta', 103635), ('intact', 101326), ('angle', 96657), ('active', 94486), ('abnormal', 93758), ('ap', 88717), ('pleural', 88118), ('cardiomegaly', 87718), ('tube', 87283), ('cage', 83502), ('film', 80921), ('lower', 79526)

In [7]:
report_corpus.collocations(freq_rank)

status post; heart size; thoracic cage; lung lesion; lung field;
pleural effusion; costophrenic angle; normal heart; active lung;
normal limit; within normal; definite abnormal; abnormal change;
generally intact; aortic arch; soft tissue; free air; cxr show;
subphrenic free; without compression; ventricular enlargement; midline
without; endotracheal tube; lower lung; bony thorax; definite lung;
aortic knob; regular chest; tissue shadow; tube insertion; left
ventricular; erect chest; lung marking; pleural thickening; central
venous; tortuous aorta; bilateral costophrenic; deep sulcus; film
show; essential intact; sulcus sign; nonspecific finding; tip placed;
clinical manifestation; bony thoracic; spur formation; chest routine;
nasogastric tube; abnormal contour; right lower


In [8]:
num_samples = 10

for i in np.random.randint(df_raw.shape[0], size=num_samples):
    print('\n\n--------------Original Report Text ' + str(i) + ' -------------------\n')
    print(df_raw.iloc[[i],[15]].values)

    print('\n\n--------------Processed Report Text ' + str(i) + ' -------------------\n')
    print(df_report_only.iloc[[i],[1]].values)
    



--------------Original Report Text 350880 -------------------

[['Supine chest AP view was taken:$* S/P pigtail catheter insertion on the right side.$* S/P VP shunting appearance.$* S/P endotracheal tube intubation with its tip placed above the carina.$* S/P NG tube insertion.$* Alveolar infiltrates are noted in the both lungs.$* Bilateral pleural effusions suggested.$* The heart size is within normal limit.$* Tortuosity of the thoracic aorta with curvilinear calcification at the aortic knob is noted.$']]


--------------Processed Report Text 350880 -------------------

[['supine chest ap view wa taken status post pigtail catheter insertion on the right side status post vp shunting appearance status post endotracheal tube intubation with it tip placed above the carina status post ng tube insertion alveolar infiltrates are noted in the both lung bilateral pleural effusion suggested the heart size is within normal limit tortuosity of the thoracic aorta with curvilinear calcification at

In [9]:
# String search for 'blunt' and label all of them positive for pleural effusion

df_blunt = df_report_only[df_report_only['Report'].str.contains('blunt')]

df_blunt['PE Label'] = 1

print('Sample counts after applying filter: {}'.format(df_blunt.shape[0]))

Sample counts after applying filter: 13703


/Users/yenchou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [10]:
# String search for 'sharp' and label all of them negative for pleural effusion

df_sharp = df_report_only[df_report_only['Report'].str.contains('sharp')]

df_sharp['PE Label'] = 0

print('Sample counts after applying filter: {}'.format(df_sharp.shape[0]))

Sample counts after applying filter: 45171


/Users/yenchou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [11]:
# Search for most common patterns of negative description regarding pleural effusion in our current text corpus

PE_neg_match = TokenSearcher(report_corpus).findall(r'<no><.*>{,2}<pleural><effusion>')

PE_neg_phrase = [" ".join(phrase) for phrase in PE_neg_match]

FreqDist(PE_neg_phrase).most_common()

[('no evidence of pleural effusion', 17138),
 ('no apparent pleural effusion', 3843),
 ('no significant pleural effusion', 946),
 ('no obvious pleural effusion', 473),
 ('no cardiomegaly mild pleural effusion', 377),
 ('no prominent pleural effusion', 227),
 ('no pleural effusion', 183),
 ('no cardiomegaly moderate pleural effusion', 64),
 ('no cardiomegaly minimal pleural effusion', 35),
 ('no visible pleural effusion', 22),
 ('no visible left pleural effusion', 21),
 ('no visible right pleural effusion', 10),
 ('no definite pleural effusion', 6),
 ('no cardiomegaly massive pleural effusion', 6),
 ('no visible bilateral pleural effusion', 5),
 ('no significant right pleural effusion', 5),
 ('no left pleural effusion', 4),
 ('no pneumothorax bilateral pleural effusion', 3),
 ('no obvious cardiomegaly pleural effusion', 3),
 ('no cardiomegaly left pleural effusion', 3),
 ('no obvoius pleural effusion', 3),
 ('no right pleural effusion', 3),
 ('no obvious left pleural effusion', 3),
 ('n

In [12]:
# Filtered text corpus for negative descriptions most commonly used, extracting and labelling negative pleural effusion cases with high confidence

PE_neg_keyword = ['no evidence of pleural effusion','no apparent pleural effusion','no obvious pleural effusion','no significant pleural effusion','no pleural effusion','no prominent pleural effusion','no visible pleural effusion','no visible left pleural effusion']

pat = "|".join(PE_neg_keyword)

df_PE_neg = df_report_only[df_report_only['Report'].str.contains(pat)]

df_PE_neg['PE Label'] = 0

print('Sample counts after applying filter: {}'.format(df_PE_neg.shape[0]))

Sample counts after applying filter: 22851


/Users/yenchou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [13]:
# Removing negative reports from all the 'pleural effusion' containing reports, and label the remaining cases positive for pleural effusion

df_PE_pos = df_report_only[df_report_only['Report'].str.contains('effusion') & ~(df_report_only['Report'].str.contains(pat))]

df_PE_pos['PE Label'] = 1

print('Sample counts after applying filter: {}'.format(df_PE_pos.shape[0]))

Sample counts after applying filter: 37017


/Users/yenchou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [14]:
for i in np.random.randint(df_PE_pos.shape[0], size=num_samples):
    print('\n')
    print(df_PE_pos.iloc[[i],[1]].values)




[['pa view of chest show no active lung lesion no cardiomegaly mild pleural effusion left old rib fracture left degenerative change of the tspine ng tube in place dilated bowel gas']]


[['film of portable chest ap supine tortuous of aorta exaggeration marking over both lung and evidence of pneumonia with pleural effusion over right lower post pigtail drainage are showed']]


[['the chest ap view show status post endotracheal tube insertion with fair position status post esopharyngealgastric tube insertion no subphrenic air or pneumothorax mild blunting of right cp angle suspect pleural effusion no specific lung lesion']]


[['chest ap show enlarged heart shadow tortuous and calcified aorta left pleural effusion status post vascular stent insertion at the abdominal aorta status post pigtail insertion at the r t abdomen']]


[['supine cxr show no deep sulcus sign to suggest pneumothorax cardiomegaly and tortuous aorta with calcification at aortic arch status post ng intubation there i

In [19]:
df_output = df_blunt.append([df_PE_pos, df_sharp,df_PE_neg])

df_output.to_csv('data//Pleural Effusion List(labels generated by reports)')

df_output

,ACCNO,Report,PE Label
10,RA04A12974381130,follow up supine cxr show no active lung lesio...,1
59,RA06529497620064,chest ap show bilateral perihilar infiltration...,1
97,RA06724349590003,chest pa show mild left ventricular enlargemen...,1
151,RA03110848230001,the pa chest radiograph show some alveolar inf...,1
156,RA05802458340016,erect chest pa view show right apical pleural ...,1
160,RA07403944030010,chest pa view bilateral pedicle of vertebra we...,1
197,RA05223882100001,supine cxr show increased lung marking and emp...,1
209,RA07313518960002,chest pa show mild left ventricular enlargemen...,1
259,RA06218232400016,chest xray show normal cardiac size atheroscle...,1
269,RA06B26595300077,chest ap show infiltration at right lung blunt...,1


In [59]:
PE_pos_long_report = [text.replace('pleural effusion','*** PLEURAL EFFUSION ***').replace('blunting','*** BLUNTING ***') for text in (df_output[(df_output['PE Label'] == 1)]['Report']) if len(text.split())> 60]

pd.Series(PE_pos_long_report).to_csv('PE Positive Reports Highlighted.csv')